In [3]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('deepfake_model.h5')

# Check model input shape
print("Model Input Shape:", model.input_shape)

img_size = 128  # Adjust to match your model input size

# Path to folder with test videos
video_folder = r"E:\archive\video_batches\real"

# Optional: Test a known real image to verify model sanity
def test_real_image(model, img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)  # (1, 128, 128, 3)
    confidence = model.predict(img, verbose=0)[0][0]
    print(f"Test Image Confidence (FAKE): {confidence:.4f}")

# Frame-by-frame prediction function with smoothing
def predict_from_video(video_path, every_n_frames=5, threshold=0.9, show_each_confidence=False):
    cap = cv2.VideoCapture(video_path)
    frame_id = 0
    predictions = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_id % every_n_frames == 0:
            frame_resized = cv2.resize(frame, (img_size, img_size))
            frame_resized = frame_resized.astype("float32") / 255.0
            frame_input = np.expand_dims(frame_resized, axis=0)  # (1, 128, 128, 3)
            frame_input = np.expand_dims(frame_input, axis=1)    # (1, 1, 128, 128, 3)

            confidence = model.predict(frame_input, verbose=0)[0][0]
            predictions.append(confidence)

            if show_each_confidence:
                print(f"Frame {frame_id}: Confidence (FAKE) = {confidence:.4f}")

        frame_id += 1

    cap.release()

    if not predictions:
        return "UNKNOWN", 0.0, 0

    # Apply a smoothing technique: average of all predictions
    avg_confidence = np.mean(predictions)

    # Introduce dynamic threshold or experiment with adjusting it
    label = "FAKE" if avg_confidence >= threshold else "REAL"

    # Optionally adjust threshold dynamically based on confidence distribution
    if avg_confidence < 0.5:
        label = "REAL"
    
    return label, avg_confidence, len(predictions)

# Loop through videos and test with the 90% threshold
for video_name in os.listdir(video_folder):
    video_path = os.path.join(video_folder, video_name)
    if video_path.endswith('.mp4'):
        print(f"\n🎥 Processing: {video_name}")

        # Test with a 90% threshold
        label, confidence, total_frames = predict_from_video(video_path, threshold=0.9)
        print(f"Threshold = 0.9 → Verdict: {label} | Confidence: {confidence:.4f} | Frames: {total_frames}")


Model Input Shape: (None, 1, 128, 128, 3)

🎥 Processing: 01__exit_phone_room.mp4


KeyboardInterrupt: 

In [4]:
import os

# Define the path
folder_path = r"E:\Dataset\Validation\Fake"

# Number of images to delete
delete_count = 9000

# Image file extensions
image_extensions = ('.jpg', '.jpeg', '.png')

# List all matching files starting with "real_" and ending with a valid image extension
image_files = [file for file in os.listdir(folder_path)
               if file.lower().startswith("real_") and file.lower().endswith(image_extensions)]

# Sort the files for consistency
image_files.sort()

# Delete the first 45,000 files
for idx, img in enumerate(image_files[:delete_count]):
    file_path = os.path.join(folder_path, img)
    try:
        os.remove(file_path)
        print(f"Deleted [{idx+1}/{delete_count}]: {img}")
    except Exception as e:
        print(f"Failed to delete {img}: {e}")

print("Deletion process completed.")


Deletion process completed.
